# 상관분석

* internal_metrics() 이상치를 한번 파악하려고 봤더니 상자그림이 이상하게 그려졌다. 25
* knob 이상치 제거할 필요 없다. 랜덤생성했기 때문에 14 (+ 범주형은 10개)

1. 데이터 전처리
2. 상관분석 함수
3. activedefrag 있는 경우
4. 전체 knob
5. activedefrag 없는 경우
6. pointbeir~


len = 17
cluster0
hz
lru_clock
used_memory_peak
used_memory_overhead
allocator_allocated
allocator_active
maxmemory
maxmemory_human
allocator_frag_bytes
rss_overhead_ratio
rss_overhead_bytes
rdb_last_save_time
instantaneous_ops_per_sec
instantaneous_input_kbps
instantaneous_output_kbps
used_cpu_sys
used_cpu_user

len = 7
cluster1
used_memory_rss
used_memory_rss_human
allocator_resident
allocator_rss_ratio
allocator_rss_bytes
mem_fragmentation_ratio
mem_fragmentation_bytes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# internal metrics 분류 결과
cluster0 =[ 
'uptime_in_seconds',
'lru_clock',
'used_memory_peak_perc',
'used_memory_dataset_perc',
'allocator_frag_ratio',
'mem_not_counted_for_evict',
'mem_aof_buffer',
'rdb_last_save_time',
'rdb_last_bgsave_time_sec',
'rdb_current_bgsave_time_sec',
'rdb_last_cow_size',
'aof_current_size',
'aof_base_size',
'latest_fork_usec',
'used_cpu_sys',
'used_cpu_user',
'used_cpu_sys_children',
'used_cpu_user_children',
]

cluster1 = [
'hz',
'used_memory',
'used_memory_human',
"used_memory_rss",
'used_memory_rss_human',
'used_memory_peak',
'used_memory_peak_human',
'used_memory_overhead',
'used_memory_dataset',
'allocator_allocated',
'allocator_active',
'allocator_resident',
'maxmemory',
'maxmemory_human',
'allocator_frag_bytes',
'allocator_rss_ratio',
'allocator_rss_bytes',
'rss_overhead_ratio',
'rss_overhead_bytes',
'mem_fragmentation_ratio',
'mem_fragmentation_bytes',
'rdb_changes_since_last_save',
'instantaneous_ops_per_sec',
'instantaneous_input_kbps',
'instantaneous_output_kbps',
]

knob_list = [
    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",  ##yes
    "hz",
    "dynamic-hz",
    
    # appendonly 제외
    'appendfsync',
    'auto-aof-rewrite-percentage',
    'auto-aof-rewrite-min-size',
    'no-appendfsync-on-rewrite',
    'aof-rewrite-incremental-fsync',
    'aof-use-rdb-preamble',
    
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",   ##"volatile-lru"
    "maxmemory-policy",
    "maxmemory-samples",
    
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [3]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
# internal_csv = pd.read_csv('./data/internal_metrcis0.csv')

file_dir = '../data/'
file_name = 'internal_metrics_r10000_aof2.csv'

internal_csv = pd.read_csv(file_dir+file_name)

# 1250 개 추출
internal_csv = internal_csv.iloc[0:1250,:]
internal_csv.shape

(1000, 131)

In [4]:
# role == master 인 부분 제외
idx_role_null = internal_csv['role'] == 'master'
internal_csv = internal_csv[idx_role_null]
internal_csv.shape

(1000, 131)

In [6]:
# 데이터 불러오기 knob_list
# knobs = pd.read_csv('./data/result_config0.csv')

file_dir = '../data/'
file_name = 'result_config_r10000_aof.csv'

knobs = pd.read_csv(file_dir + file_name)
# 1250 개 추출
knobs = knobs.iloc[0:1000,:]
knobs.shape

(1000, 32)

In [7]:
# role == master 인 부분 제외
knobs = knobs[idx_role_null]
knobs.shape

(1000, 32)

In [8]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

## 데이터 전처리

In [9]:
# 클러스터 합치기
cluster = cluster0+cluster1

In [10]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)

### 결측치 확인

In [11]:
# internal_csv 결측치 확인 
total_internal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   uptime_in_seconds            1000 non-null   int64  
 1   lru_clock                    1000 non-null   int64  
 2   used_memory_peak_perc        1000 non-null   float64
 3   used_memory_dataset_perc     1000 non-null   float64
 4   allocator_frag_ratio         1000 non-null   float64
 5   mem_not_counted_for_evict    1000 non-null   int64  
 6   mem_aof_buffer               1000 non-null   int64  
 7   rdb_last_save_time           1000 non-null   int64  
 8   rdb_last_bgsave_time_sec     1000 non-null   int64  
 9   rdb_current_bgsave_time_sec  1000 non-null   int64  
 10  rdb_last_cow_size            1000 non-null   int64  
 11  aof_current_size             1000 non-null   int64  
 12  aof_base_size                1000 non-null   int64  
 13  latest_fork_usec   

In [12]:
# # # throughput 결측치 제거
# total_internal_df=total_internal_df.dropna()

In [13]:
# maxmemory 처리 knob
knobs['maxmemory'] = knobs['maxmemory'].str.strip('mb')  # 단위 자르기
knobs['maxmemory'] = pd.to_numeric(knobs['maxmemory'])  # 숫자형으로 바꾸기

# auto-aof-rewrite-min-size knob 단위(mb) 처리
knobs['auto-aof-rewrite-min-size']  = knobs['auto-aof-rewrite-min-size'].str.strip('mb')
knobs['auto-aof-rewrite-min-size'] = pd.to_numeric(knobs['auto-aof-rewrite-min-size'])

### activedefrag 처리
* subTotal_knobs_Active : 약 1300개 이며 activedefrag가 있는 경우 
* subTotal_knobs_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* subTotal_knobs_T : 2500개 이며 activedefrag 상관없이 전체 데이터


* total_internal_Active : 약 1300개 이며 activedefrag가 있는 경우 
* total_internal_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* total_internal_T : 2500개 이며 activedefrag 상관없이 전체 데이터

In [14]:
# knob_list 만 추출
subTotal_knobs = extract_data(knob_list, knobs)

In [15]:
#activedefrag knobs 분리

# activedefrag O 존재
subTotal_knobs_Active = subTotal_knobs.dropna()
Active_index = list(subTotal_knobs_Active.index)  # 인덱스

# # activedefrag X 존재하지 않는 나머지 반
subTotal_knobs_NoActive = subTotal_knobs.loc[subTotal_knobs['activedefrag'].isna()].dropna(axis=1)
NoActive_index = list(subTotal_knobs_NoActive.index)  # 인덱스

# activedefrag O and X 전체 
subTotal_knobs_T = subTotal_knobs.dropna(axis=1)
total_index = list(subTotal_knobs_T.index)   # 인덱스

In [16]:
# knobs에서 타입별로 분리 (수치, 불리언, 불리언아닌 문자)

def varType(subTotal_knobs):
    object_var = []
    numerical_var = []  # 문자, 숫자 타입
    
    for idx in subTotal_knobs.dtypes.index:
        if subTotal_knobs.dtypes[idx] == 'O':  # 문자일 경우
            object_var.append(idx)
        else: 
            numerical_var.append(idx)  # 숫자일 경우

    boolean_var = []
    booleanNo_var = []  # 2개, 2개이상

    for col in object_var:
        col_count = len(subTotal_knobs[col].unique())

        if col_count == 2:  # 2개일 경우
            boolean_var.append(col)
        elif col_count > 2:  # 2개 이상일 경우
            booleanNo_var.append(col)
    
    return numerical_var, boolean_var, booleanNo_var

# 수치, 불리언, 불리언아닌것 (전체)
numerical_var, boolean_var, NobooleanNo_var = varType(subTotal_knobs_T)

# 수치, 불리언, 불리언아닌것 (activedefrag)
numericalActive_var, booleanActive_var, NobooleanNoActive_var = varType(subTotal_knobs_Active)

# 수치, 불리언, 불리언아닌것 (activedefrag 없는것)
numericalNoActive_var, booleanNoActive_var, NobooleanNoActive_var = varType(subTotal_knobs_NoActive)


In [17]:
# internal에서 분리 

# activedefrag O
total_internal_Active = total_internal_df.loc[Active_index]

# activedefrag X
total_internal_NoActive = total_internal_df.loc[NoActive_index]

# activedefrag O and X
total_internal_T = total_internal_df.loc[total_index]

---

## 상관분석 함수

In [18]:
def get_outlier(df=None, column=None, weight=1.5):
    
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [19]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


# knob하나 internal 여러개 상관계수 구하기

# pearson
## 모수 검정
def cal_pearsonr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pearsonr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = kendalltau(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = spearmanr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df


# knob하나 internal 여러개 상관계수 구하기 이상치 제거 버전
# pearson
## 모수 검정
def cal_pearsonr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pearsonr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = kendalltau(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

    
# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = spearmanr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])



In [20]:
# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pearson')
    df = cal_pearsonr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('kendalltau')
    df = cal_kendalltau(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('spearman')
    df = cal_spearmanr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

def extract_pvalue_outlier(knobs, internal_df):
    print('pearson')
    cal_pearsonr_outlier(knobs, internal_df)
    print()
    
    print('kendalltau')
    cal_kendalltau_outlier(knobs, internal_df)
    print()
    
    print('spearman')
    cal_spearmanr_outlier(knobs, internal_df)
    print()
    

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)
        
        
# knob 리스트 각각 별로 extract_pvalue 수행 이상치 제거 
def extract_pvalue_knob_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue_outlier(subTotal_knobs[knob], total_internal)

---

## activedefrag 상관계수
* subTotal_knobs_Active
* total_internal_Active

### numericalActive_var :
* active-defrag-threshold-lower 
* active-defrag-threshold-upper 
* active-defrag-cycle-min
* active-defrag-cycle-max

In [21]:
numericalActive_var

['hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz',
 'auto-aof-rewrite-percentage',
 'auto-aof-rewrite-min-size',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples']

In [22]:
numericalActive_var
numericalActive_var_new = numericalActive_var[5:9]
numericalActive_var_new

['active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max']

In [23]:
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson
                           corr   p-value
used_memory_rss       -0.093445  0.040296
used_memory_rss_human -0.093404  0.040384

kendalltau
                            corr   p-value
used_memory_peak_perc   0.088642  0.012049
used_memory_rss        -0.068453  0.028818
used_memory_rss_human  -0.069836  0.027951
used_memory_peak       -0.103621  0.002327
used_memory_peak_human -0.124196  0.000553

spearman
                            corr   p-value
used_memory_peak_perc   0.113908  0.012334
used_memory_rss        -0.099451  0.029025
used_memory_rss_human  -0.100001  0.028144
used_memory_peak       -0.138050  0.002386
used_memory_peak_human -0.155922  0.000592

###active-defrag-threshold-upper###
pearson
                          corr   p-value
allocator_frag_ratio -0.097563  0.032234
used_memory_human     0.093344  0.040515
allocator_allocated   0.090928  0.046017

kendalltau
                      corr   p-value
used_memory_human  0.08369  0.0257

In [24]:
# cluster 별 knob들의 상관계수 합  
active_result = pd.DataFrame(index = numericalActive_var_new)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numericalActive_var_new:
        df = cal_pearsonr(subTotal_knobs_Active[knob], total_internal_Active.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    active_result['cluster'+str(idx)] = cluster_val

In [25]:
active_result

,cluster0,cluster1
active-defrag-threshold-lower,0.031139,0.048150
active-defrag-threshold-upper,0.047910,0.041213
active-defrag-cycle-min,0.033025,0.039090
active-defrag-cycle-max,0.057954,0.052319


In [26]:
# knob이 어떤 클러스터에 속하는지 확인 가능 함수
def get_cluster(df):
    cluster_list = df.columns.tolist()  # [cluster0, cluster1]
    for knob_index in list(df.index):
        max_index = np.argmax(df.loc[knob_index,:].values)
        print(knob_index+' : ', end='')
        print(cluster_list[max_index])    

In [28]:
get_cluster(active_result)

active-defrag-threshold-lower : cluster1
active-defrag-threshold-upper : cluster0
active-defrag-cycle-min : cluster1
active-defrag-cycle-max : cluster0


### 이상치 제거 후 상관계수

In [29]:
# extract_pvalue_knob_outlier(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

---

## 전체 knob
* subTotal_knobs_T
* total_internal_T

### numerical_var

In [30]:
numerical_var

['hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz',
 'auto-aof-rewrite-percentage',
 'auto-aof-rewrite-min-size',
 'maxmemory',
 'maxmemory-samples']

In [31]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)

###hash-max-ziplist-entries###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                              corr   p-value
used_memory_dataset_perc  0.064536  0.010884
allocator_allocated      -0.044087  0.042675

spearman
                              corr   p-value
used_memory_dataset_perc  0.080826  0.010560
allocator_allocated      -0.062974  0.046494

###hash-max-ziplist-value###
pearson
                      corr   p-value
aof_current_size -0.075339  0.017179
aof_base_size    -0.070003  0.026853
maxmemory         0.070267  0.026283

kendalltau
                      corr   p-value
aof_current_size -0.059541  0.021374
maxmemory         0.046931  0.030466
maxmemory_human   0.061528  0.004553

spearman
                      corr   p-value
aof_current_size -0.072806  0.021307
maxmemory         0.068772  0.029658
maxmemory_human   0.090072  0.004364

###hz###
pearson
                                 corr       p-value
uptime_in_seconds            0.080269  1.110

In [32]:
subTotal_knobs_T  # knobs에서 aof 관련 제거한 것
total_internal_T  # 전체

,uptime_in_seconds,lru_clock,used_memory_peak_perc,used_memory_dataset_perc,allocator_frag_ratio,mem_not_counted_for_evict,mem_aof_buffer,rdb_last_save_time,rdb_last_bgsave_time_sec,rdb_current_bgsave_time_sec,...,allocator_rss_ratio,allocator_rss_bytes,rss_overhead_ratio,rss_overhead_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes,rdb_changes_since_last_save,instantaneous_ops_per_sec,instantaneous_input_kbps,instantaneous_output_kbps
0,6,4510577,77.29,56.99,1.14,4,8,1631900523,-1,-1,...,2.74,3612672,0.91,-483328,2.98,3457280,20000,7530,551.52,36.77
1,6,4510596,96.70,56.98,1.14,220,224,1631900542,-1,-1,...,2.45,2949120,0.95,-266240,2.77,3011584,20000,16533,1210.99,80.74
2,1145,4511748,96.70,56.98,1.14,220,224,1631901458,0,-1,...,2.43,2912256,0.93,-327680,2.65,2871336,4185,2,0.15,0.01
3,6,4511759,96.70,56.98,1.14,220,224,1631901706,-1,-1,...,3.34,4968448,0.67,-2371584,2.58,2891832,20000,2500,183.11,12.21
4,1081,4512847,96.70,56.98,1.14,220,224,1631902621,0,-1,...,3.47,5013504,0.65,-2490368,2.61,2809896,3178,19,1.47,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6,4870070,96.70,56.98,1.14,220,224,1632260016,-1,-1,...,2.52,3088384,0.92,-385024,2.78,3032064,20000,6250,457.76,30.52
996,6,4870082,96.70,56.98,1.14,220,224,1632260028,-1,-1,...,2.46,2969600,0.94,-299008,2.69,2957352,20000,16856,1234.59,82.31
997,6,4870094,96.70,56.98,1.14,156,160,1632260040,-1,-1,...,2.43,2965504,0.93,-356352,2.68,2936936,20000,15209,1114.01,74.27
998,1088,4871188,98.92,56.98,1.14,220,224,1632260955,0,-1,...,3.45,4968448,0.66,-2383872,2.65,2871336,3429,5,0.39,0.03


In [33]:
# cluster 별 knob들의 상관계수 합  
numerical_result = pd.DataFrame(index = numerical_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numerical_var:
        df = cal_spearmanr(subTotal_knobs_T[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    numerical_result['cluster'+str(idx)] = cluster_val

In [34]:
numerical_result

,cluster0,cluster1
hash-max-ziplist-entries,0.021664,0.022531
hash-max-ziplist-value,0.025705,0.027385
hz,0.097621,0.148766
auto-aof-rewrite-percentage,0.021339,0.043374
auto-aof-rewrite-min-size,0.034780,0.029458
maxmemory,0.018350,0.083528
maxmemory-samples,0.031354,0.028773


In [36]:
get_cluster(numerical_result)

hash-max-ziplist-entries : cluster1
hash-max-ziplist-value : cluster1
hz : cluster1
auto-aof-rewrite-percentage : cluster1
auto-aof-rewrite-min-size : cluster0
maxmemory : cluster1
maxmemory-samples : cluster0


In [37]:
# numerical_result

In [38]:
# extract_pvalue_knob_outlier(numerical_var, subTotal_knobs_T,total_internal_T)

---

## Activedefrag 없는 knob들
* subTotal_knobs_Active
* total_internal_Active
active인거는 신경 안써도 된다.
### numericalActive_var

In [39]:
# numericalActive_var

In [40]:
# extract_pvalue_knob(numericalActive_var, subTotal_knobs_Active,total_internal_Active)

## other knob
* subTotal_knobs_NoActive
* total_internal_NoActive
active인거는 신경 안써도 된다.
### numericalNoActive_var

In [41]:
# numericalNoActive_var

In [42]:
# extract_pvalue_knob(numericalNoActive_var, subTotal_knobs_NoActive,total_internal_NoActive)

---

## pointbiserialr 분석

In [43]:
from scipy.stats import pointbiserialr
# pointbiserialr
## yes,no 데이터와 연속형 데이터 상관분석
def cal_pointbiserialr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pointbiserialr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pointbierialr')
    df = cal_pointbiserialr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

    
# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


# 이상치 제거 버전
def cal_pointbiserialr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pointbiserialr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue_outlier(knobs, internal_df):
    print('pointbierialr')
    cal_pointbiserialr(knobs, internal_df)
    print()

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### boolean_var

In [44]:
boolean_var

['activerehashing',
 'dynamic-hz',
 'no-appendfsync-on-rewrite',
 'aof-rewrite-incremental-fsync',
 'aof-use-rdb-preamble',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del']

In [45]:
#  yes, no -> 1, 0으로 변경
subTotal_knobs_T_bool = subTotal_knobs_T[boolean_var]  ## boolean 변수 df 추출
subTotal_knobs_T_point = subTotal_knobs_T_bool.replace(['yes', 'no'], [1,0])  ## 1, 0 변경

In [46]:
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)

###activerehashing###
pointbierialr
                       corr   p-value
rdb_last_cow_size -0.063045  0.046245

###dynamic-hz###
pointbierialr
                               corr   p-value
maxmemory                 -0.083693  0.008098
instantaneous_ops_per_sec  0.083743  0.008060
instantaneous_input_kbps   0.083743  0.008061
instantaneous_output_kbps  0.083743  0.008060

###no-appendfsync-on-rewrite###
pointbierialr
                                 corr       p-value
uptime_in_seconds           -0.235222  4.881466e-14
used_memory_dataset_perc     0.090066  4.366647e-03
rdb_last_bgsave_time_sec    -0.079788  1.160309e-02
rdb_current_bgsave_time_sec  0.066698  3.495505e-02
used_cpu_sys                -0.227158  3.604750e-13
used_cpu_user               -0.219760  2.113230e-12
used_cpu_sys_children       -0.156293  6.806592e-07
used_cpu_user_children      -0.162786  2.268710e-07
used_memory                 -0.075440  1.703078e-02
used_memory_overhead        -0.077849  1.379934e-02
allocat

In [47]:
# cluster 별 knob들의 상관계수 합
bool_result = pd.DataFrame(index = boolean_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in boolean_var:
        df = cal_pointbiserialr(subTotal_knobs_T_point[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    bool_result['cluster'+str(idx)] = cluster_val

In [48]:
bool_result

,cluster0,cluster1
activerehashing,0.026440,0.014465
dynamic-hz,0.028477,0.033620
no-appendfsync-on-rewrite,0.085262,0.049673
aof-rewrite-incremental-fsync,0.030025,0.015827
aof-use-rdb-preamble,0.016700,0.026803
lazyfree-lazy-eviction,0.031023,0.032609
lazyfree-lazy-expire,0.030405,0.013269
lazyfree-lazy-server-del,0.025639,0.017119


In [49]:
get_cluster(bool_result)

activerehashing : cluster0
dynamic-hz : cluster1
no-appendfsync-on-rewrite : cluster0
aof-rewrite-incremental-fsync : cluster0
aof-use-rdb-preamble : cluster1
lazyfree-lazy-eviction : cluster1
lazyfree-lazy-expire : cluster0
lazyfree-lazy-server-del : cluster0


In [52]:
# #이상치 제거
# extract_pvalue_knob_point_outlier(boolean_var, subTotal_knobs_T_point, total_internal_T)

---

In [53]:
# extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)
# extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)
# extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)